In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

2025-02-02 17:48:30.273565: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-02 17:48:30.281052: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738511310.291871   36177 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738511310.295073   36177 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-02 17:48:30.305839: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
train_dir = "/mnt/c/Users/iosif/Desktop/Python/Tensorflow/101_food_classes_full/train"
test_dir = "/mnt/c/Users/iosif/Desktop/Python/Tensorflow/101_food_classes_full/validation"
#-------------------------
train_data = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                    image_size=(224,224),
                                                    label_mode="categorical",
                                                    shuffle=True)
test_data = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                    image_size=(224,224),
                                                    label_mode="categorical",
                                                    shuffle=True)

Found 80800 files belonging to 101 classes.


2025-02-02 17:49:04.352090: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (100)


Found 20200 files belonging to 101 classes.


# EfficientNet

In [3]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomHeight(0.1),
    tf.keras.layers.RandomWidth(0.1)
])

In [5]:
base_model = tf.keras.applications.EfficientNetB0(input_shape=(224,224,3), include_top=False)
base_model.trainable = False

inputs = tf.keras.Input(shape=(224,224,3))
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(101, activation="softmax")(x)
efi_big = tf.keras.Model(inputs, outputs)

efi_big.compile(loss = "categorical_crossentropy",
                  optimizer = "Adam",
                  metrics=["accuracy"])

stop_early = tf.keras.callbacks.EarlyStopping(patience=3)
hist_1 = efi_big.fit(epochs=100,
                       x=train_data,
                       validation_data=test_data,
                       steps_per_epoch=len(train_data),
                       validation_steps=int(0.2*len(test_data)),
                       callbacks=stop_early)

Epoch 1/100
  28/2525 ━━━━━━━━━━━━━━━━━━━━ 11:10 268ms/step - accuracy: 0.0149 - loss: 4.6535

KeyboardInterrupt: 

In [ ]:
initial_epoch = hist_1.epoch[-1] + 1

base_model.trainable = True

for layers in base_model.layers[:-10]:
    layers.trainable = False

efi_big.compile(loss="categorical_crossentropy",
                  optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001),
                  metrics = ["accuracy"])

hist_1_fine = efi_big.fit(epochs=100,
                            initial_epoch=initial_epoch,
                            x=train_data,
                            validation_data=test_data,
                            steps_per_epoch=len(train_data),
                            validation_steps=len(test_data),
                            callbacks=stop_early)

Epoch 12/100


E0000 00:00:1737702321.990647  183200 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1_1/efficientnetb0_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


2525/2525 ━━━━━━━━━━━━━━━━━━━━ 110s 42ms/step - accuracy: 0.6511 - loss: 1.3255 - val_accuracy: 0.6802 - val_loss: 1.2717
Epoch 13/100
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 103s 41ms/step - accuracy: 0.6969 - loss: 1.1304 - val_accuracy: 0.6936 - val_loss: 1.2107
Epoch 14/100
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 103s 41ms/step - accuracy: 0.7117 - loss: 1.0603 - val_accuracy: 0.6984 - val_loss: 1.1845
Epoch 15/100
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 103s 41ms/step - accuracy: 0.7277 - loss: 1.0002 - val_accuracy: 0.7029 - val_loss: 1.1686
Epoch 16/100
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 103s 41ms/step - accuracy: 0.7399 - loss: 0.9473 - val_accuracy: 0.7045 - val_loss: 1.1597
Epoch 17/100
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 103s 41ms/step - accuracy: 0.7446 - loss: 0.9202 - val_accuracy: 0.7058 - val_loss: 1.1549
Epoch 18/100
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 103s 41ms/step - accuracy: 0.7553 - loss: 0.8850 - val_accuracy: 0.7078 - val_loss: 1.1452
Epoch 19/100
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 104s 41ms/step - accurac

# RezNet

In [6]:
data_augmentation_reznet = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomHeight(0.1),
    tf.keras.layers.RandomWidth(0.1),
    tf.keras.layers.Rescaling(scale=(1./255.))
])

In [7]:
base_model_R = tf.keras.applications.ResNet50(include_top=False, input_shape=(224,224,3))
base_model_R.trainable = False

inputs = tf.keras.Input(shape=(224,224,3))
xr = data_augmentation(inputs)
xr = base_model(xr, training = False)
xr = tf.keras.layers.GlobalAveragePooling2D()(xr)
xr = tf.keras.layers.Dropout(0.2)(xr)
xr = tf.keras.layers.Dense(512, activation="relu")(xr)
outputs = tf.keras.layers.Dense(101, activation="softmax")(xr)
rez_big = tf.keras.Model(inputs, outputs)

rez_big.compile(loss="categorical_crossentropy",
                  optimizer = tf.keras.optimizers.Adam(),
                  metrics=["accuracy"])

stop_early_rez = tf.keras.callbacks.EarlyStopping(patience=3)

hist_rez = rez_big.fit(epochs=100,
                           x=train_data,
                           validation_data=test_data,
                           steps_per_epoch=len(train_data),
                           validation_steps=int(0.25*len(test_data)),
                           callbacks=stop_early_rez)

Epoch 1/100


E0000 00:00:1737703779.712958  183200 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_2_1/efficientnetb0_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


2525/2525 ━━━━━━━━━━━━━━━━━━━━ 102s 38ms/step - accuracy: 0.4816 - loss: 2.1038 - val_accuracy: 0.6446 - val_loss: 1.3387
Epoch 2/100
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 95s 38ms/step - accuracy: 0.6221 - loss: 1.4430 - val_accuracy: 0.6812 - val_loss: 1.2275
Epoch 3/100
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 95s 38ms/step - accuracy: 0.6534 - loss: 1.3010 - val_accuracy: 0.6998 - val_loss: 1.1668
Epoch 4/100
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 96s 38ms/step - accuracy: 0.6753 - loss: 1.2021 - val_accuracy: 0.7009 - val_loss: 1.1813
Epoch 5/100
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 96s 38ms/step - accuracy: 0.6932 - loss: 1.1250 - val_accuracy: 0.6926 - val_loss: 1.1927
Epoch 6/100
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 96s 38ms/step - accuracy: 0.7060 - loss: 1.0612 - val_accuracy: 0.7073 - val_loss: 1.1862


In [8]:
initail_epoch_r = hist_rez.epoch[-1] + 1

base_model_R.trainable = True

for layer in rez_big.layers[:-20]:
    layer.trainable = False

base_model_R.compile(loss="categorical_crossentropy",
                     optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001),
                     metrics=["accuracy"])

hist_rez_fine = rez_big.fit(epochs=100,
                                  initial_epoch=initail_epoch_r,
                                  x=train_data,
                                  validation_data=test_data,
                                  steps_per_epoch = len(train_data),
                                  validation_steps = len(test_data),
                                  callbacks=stop_early_rez)

Epoch 7/100
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 107s 42ms/step - accuracy: 0.7210 - loss: 1.0105 - val_accuracy: 0.6895 - val_loss: 1.2447
Epoch 8/100
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 110s 43ms/step - accuracy: 0.7304 - loss: 0.9726 - val_accuracy: 0.6862 - val_loss: 1.2818
Epoch 9/100
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 108s 43ms/step - accuracy: 0.7398 - loss: 0.9279 - val_accuracy: 0.7022 - val_loss: 1.2206
Epoch 10/100
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 110s 43ms/step - accuracy: 0.7517 - loss: 0.8836 - val_accuracy: 0.6919 - val_loss: 1.3030
Epoch 11/100
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 109s 43ms/step - accuracy: 0.7556 - loss: 0.8653 - val_accuracy: 0.6993 - val_loss: 1.2811
Epoch 12/100
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 108s 43ms/step - accuracy: 0.7617 - loss: 0.8399 - val_accuracy: 0.6985 - val_loss: 1.2925


In [12]:
efi_big_1.evaluate(test_data, verbose=1)
rez_big.evaluate(test_data, verbose=1)

632/632 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - accuracy: 0.5668 - loss: 1.7662
632/632 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - accuracy: 0.6977 - loss: 1.2886


[1.2924665212631226, 0.6984653472900391]